### 线程篇

#### 线程的两种调用形式

In [5]:
#直接调用

import threading

def say_hi(words):
    print("I say:{}".format(words))
    
#这里注意:args参数即使是一个参数也要加上后面的`,`
#如果不对Thread进行命名这系统会默认命名
thread_1=threading.Thread(target=say_hi,args=("I'm thread_1",),name='T1')
thread_2=threading.Thread(target=say_hi,args=("I'm thread_1",),name='T2')

#start()函数会调用Thread类的run()函数
thread_1.start()
thread_2.start()

print(thread_1.getName())
print(thread_2.getName())

I says:I'm thread_1
I says:I'm thread_1
T1
T2


In [8]:
#继承类
import threading
import time

class MyThread(threading.Thread):
    def __init__(self,num):
        super().__init__()
        self.num=num
    #每个线程类都必须定义run()函数
    def run(self):
        print("running on number:{}".format(self.num))
        time.sleep(3)
        
t1=MyThread(1)
t2=MyThread(2)

t1.start()
t2.start()

print("\nending....")

running on number:1running on number:2


ending....


#### Thread类的方法

* join()：在子线程完成运行之前，这个子线程的父线程将一直被阻塞。

* setDaemon(True)：

         将线程声明为守护线程，必须在start() 方法调用之前设置， 如果不设置为守护线程程序会被无限挂起。这个方法基本和join是相反的。当我们 在程序运行中，执行一个主线程，如果主线程又创建一个子线程，主线程和子线程 就分兵两路，分别运行，那么当主线程完成想退出时，会检验子线程是否完成。如 果子线程未完成，则主线程会等待子线程完成后再退出。但是有时候我们需要的是只要主线程完成了，不管子线程是否完成，都要和主线程一起退出，这时就可以用setDaemon方法啦。

* run():  线程被cpu调度后自动执行线程对象的run方法
* start():启动线程活动。
* isAlive(): 返回线程是否活动的。
* getName(): 返回线程名。
* setName(): 设置线程名。

threading模块提供的一些方法：
* threading.currentThread(): 返回当前的线程变量。
* threading.enumerate(): 返回一个包含正在运行的线程的list。正在运行指线程启动后、结束前，不包括启动前和终止后的线程。
* threading.activeCount(): 返回正在运行的线程数量，与len(threading.enumerate())有相同的结果。

####  同步锁 


多个线程操作同一个数据，可能会发生数据错乱的问题，因为一个线程拿到数据后，还没来得及对数据进行操作，cpu就有可能去执行另外一个线程，另外一个线程拿到的则是之前线程没有处理完的数据。利用同步锁可以解决该问题。

In [12]:
#同步锁实例

import threading
import time

R=threading.Lock()

num=20

def sub():
    global num
    R.acquire()
    temp=num-1
    time.sleep(0.1)
    num=temp
    print(num)
    R.release()

thread_list=[]
for i in range(20):
    thread=threading.Thread(target=sub)
    thread_list.append(thread)
    thread.start()

for thread in thread_list:
    thread.join()


19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


#### 死锁

所谓死锁： 是指两个或两个以上的进程或线程在执行过程中，因争夺资源而造成的一种互相等待的现象，若无外力作用，它们都将无法推进下去。此时称系统处于死锁状态或系统产生了死锁，这些永远在互相等待的进程称为死锁进程。

In [15]:
#有两个线程，线程1拿到了A锁，但是他只有拿到B锁才能把A锁释放，
#线程2拿到了B锁，他要拿到A锁才能释放B锁，结果这两个线程就死掉了

import threading
import time

A=threading.Lock()
B=threading.Lock()

def getB():
    A.acquire()
    print("Thread_1 get Lock A")
    time.sleep(2)
    B.acquire()
    print("Thread_1 get Lock B")
    time.sleep(2)
    A.release()
    print("Thread_1 release Lock A")
    B.release()
    time.sleep(2)
    print("Thread_1 release Lock B")
    
def getA():
    B.acquire()
    print("Thread_2 get Lock B")
    time.sleep(2)
    A.acquire()
    print("Thread_2 get Lock A")
    time.sleep(2)
    B.release()
    print("Thread_2 release Lock B")
    time.sleep(2)
    A.release()
    print("Thread_2 release Lock A")

thread_1=threading.Thread(target=getB)
thread_2=threading.Thread(target=getA)

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

print("Ending...")

Thread_1 get Lock A
Thread_2 get Lock B


KeyboardInterrupt: 

#### 递归锁

相当于把一块用了很多锁的代码块，看做一个整体，当这个代码块所有的锁都释放了，才能被其他的方法或者线程拿到锁

RLock内部共同维护着一个Lock和一个counter变量，Counter记录了acquire的次数，从而使得资源可以被多次require。

递归锁的实现原理很简单，**当加锁，递归锁的引用计数+1，解锁则-1.当引用计数为0，才能被其他线程或者方法拿到锁**。

In [2]:
import threading
import time

A=B=threading.RLock()

def getB():
    A.acquire()
    print("Thread_1 counter==1")
    time.sleep(2)
    B.acquire()
    print("Thread_1 counter==2")
    time.sleep(2)
    A.release()
    print("Thread_1 counter==1")
    time.sleep(2)
    B.release()
    print("Thread_1 counter==0")
    
def getA():
    B.acquire()
    print("Thread_2 counter==1")
    time.sleep(2)
    A.acquire()
    print("Thread_2 counter==2")
    time.sleep(2)
    B.release()
    print("Thread_2 counter==1")
    time.sleep(2)
    A.release()
    print("Thread_2 counter==0")

thread_1=threading.Thread(target=getB)
thread_2=threading.Thread(target=getA)

thread_1.start()
thread_2.start()

thread_1.join()
thread_2.join()

print('Ending...')

Thread_1 counter==1
Thread_1 counter==2
Thread_1 counter==1
Thread_1 counter==0Thread_2 counter==1

Thread_2 counter==2
Thread_2 counter==1
Thread_2 counter==0
Ending...


#### 事件Event

* class threading.Event

>An event manages a flag that can be set to true with the `set()` method and reset to false with the `clear()` method. The `wait()` method blocks(阻塞) until the flag is true. The flag is initially false.`is_set()` return True if and only if the internal flag is true

In [4]:
#set->wait->clear(由不同的调用者调用)

#threading.Event可以使一个线程等待其他线程的通知。
#其内置了一个标志，初始值为False。
#线程通过wait()方法进入等待状态，直到另一个线程调用set()方法将内置标志设置为True时，Event通知所有等待状态的线程恢复运行；
#调用clear()时重置为 False。还可以通过isSet()方法查询Envent对象内置状态的当前值。

import threading
import time

event=threading.Event()

def func():
    print('%s wait for event...'%threading.currentThread().getName())
    event.wait()
    print('%s recv the event...'%threading.currentThread().getName())
    
thread_1=threading.Thread(target=func,name='Thread_1')
thread_2=threading.Thread(target=func,name='Thread_2')

thread_1.start()
thread_2.start()

time.sleep(2)

print("MainThread set event...")
event.set()

thread_1.join()
thread_2.join()
print("Ending...")

Thread_1 wait for event...
Thread_2 wait for event...
MainThread set event...
Thread_2 recv the event...Thread_1 recv the event...

Ending...


#### 信号量

信号量用来控制线程并发数的，Semaphore管理一个内置的计数器，每当调用`acquire()`时$-1$，调用`release()`时$+1$。 计数器不能小于$0$，当计数器为$0$时，`acquire()`将阻塞线程至同步锁定状态，直到其他线程调用`release()`.

In [5]:
import threading
import time

class MyThread(threading.Thread):
    def run(self):
        #acquire()如果发生阻塞则返回False,否则返回True
        if semaphore.acquire():
            print(self.name)
            time.sleep(5)
            semaphore.release()

semaphore=threading.Semaphore(5)
thread_list=[]
for i in range(20):
    thread=MyThread()
    thread_list.append(thread)
    thread.start()
for thread in thread_list:
    thread.join()

print("Ending...")

Thread-8
Thread-9
Thread-10Thread-11

Thread-12
Thread-13Thread-14Thread-16Thread-17



Thread-15
Thread-18Thread-19Thread-20Thread-21


Thread-22

Thread-23
Thread-25Thread-26Thread-24


Thread-27
Ending...


#### FIFO同步队列

```python
import Queue
q = Queue.Queue(maxsize = 10)
```
Queue.Queue类即是一个队列的同步实现。队列长度可为无限或者有限。可通过Queue的构造函数的可选参数maxsize来设定队列长度。如果maxsize小于1就表示队列长度无限。

Python Queue模块有三种队列及构造函数:
1. Python Queue模块的FIFO队列先进先出。   class queue.Queue(maxsize)
2. LIFO类似于栈，即先进后出。               class queue.LifoQueue(maxsize)
3. 还有一种是优先级队列级别越低越先出来。        class queue.PriorityQueue(maxsize)

##### 后话

由于GIL(`Global Interpreter Lock`)的存在，python中的多线程其实并不是真正的多线程。在有些情况下多线程甚至比单线程要慢。多线程在多IO的场景下使用性能较好，在计算量密集的场景下使用多进程更优。

